In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Anime Recommender System 

![](https://manga.tokyo/wp-content/uploads/2019/08/5d64c99f43599-450x320.jpg)

************************************************************************************************************

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('../input/anime-recommendations-database/Anime_data.csv',encoding='latin')  


In [ ]:
df.describe(include='O')

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.isna().mean()

In [ ]:
df.shape


In [ ]:
red=df.iloc[:,[1,3]]
red=red.drop(df[df.Synopsis.isnull()].index,0)

red.isna().any()

In [ ]:
red['Synopsis'][30]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(stop_words='english')                                   #Removing stop words 
tf_idf_matrix = tf_idf.fit_transform(red['Synopsis'])
tf_idf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

sim = linear_kernel(tf_idf_matrix, tf_idf_matrix)
sim

In [ ]:
index = red.index
titles = red['Title']
indices = pd.Series(index, index = titles).drop_duplicates()
indices

In [ ]:
def get_recommendations(title,cosine_sim = sim):
    idx = indices[title]
    sim_scores = list(enumerate(sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    wss=[]
    for i in movie_indices:
        wss.append(red['Title'][i])
        
    return wss

In [ ]:
get_recommendations('Trigun')

Lets start with other columns.


In [ ]:
black=df.iloc[:,[1,2,4,5,6,12]]
black=black.drop(black[black.Genre.isna()].index,0)
black=black.fillna('Unknown')
black.isna().sum()

In [ ]:
import numpy as np

def to_string(x):
    if x:
        return x.replace(',',' ').replace('[','').replace(']','').replace("'",'')
    else:
        return ''

#black['Genre'] = black['Genre'].astype(str)
black['Genre'] = black['Genre'].map(lambda x: to_string(x))
black['Producer'] = black['Producer'].map(lambda x: to_string(x))
black['Studio'] = black['Studio'].map(lambda x: to_string(x))

In [ ]:
black.columns

In [ ]:
def soup(x):
    return  x['Source']+' ' +x['Genre'] + ' ' + x['Type'] + ' ' + x['Producer'] + ' ' + x['Studio'] +' ' + x['Source']

In [ ]:
black['soup'] = black.apply(soup, axis = 1)

In [ ]:
me=black['soup'].values
me[85]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()
matr=cv.fit_transform(me)
matr.shape

In [ ]:
sim_sc = linear_kernel(matr, matr)

index = black.index
titles = black['Title']
ids = pd.Series(index, index = titles).drop_duplicates()

def get_recom(title,cosine_sim = sim_sc):
    idx = indices[title]
    sim_scores = list(enumerate(sim_sc[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]
    wss=[]
    for i in anime_indices:
        wss.append(black['Title'][i])
        
    return wss

In [ ]:
get_recom('One Piece')

In [ ]:
get_recommendations('One Piece')

The first one is working better as compared to the second one.

![](https://image.slidesharecdn.com/anime-recommendation-1-180121040742/95/anime-recommendation-big-data-certification6-1-638.jpg?cb=1516507702)

#                           WORK IN PROGRESS

![](https://fiverr-res.cloudinary.com/images/q_auto,f_auto/gigs/164076617/original/103631ebfeb9ec68e2566c70525b0f762c11431a/give-you-some-anime-recommendations.jpg)